In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import glob
from src.utils import Data, markdown_html
import matplotlib.pyplot as plt
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, BasicTicker)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.palettes import Inferno256
from bokeh.plotting import figure, output_file, save

In [2]:
maryland = Data(state='MD')

INFO:Data collector:Loaded geo shape
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:Data collector:Retrieved map info
INFO:Data collector:Loaded daily COVID cases
/home/wckdouglas/codes/covid/src/utils.py:61: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 61 of the file /home/wckdouglas/codes/covid/src/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(http.content)
INFO:Data collector:Retrieved populations


In [3]:
data = maryland.geo \
    .merge(maryland.zip_map, on ='Zip', how = 'right') \
    .merge(maryland.zip_covid, on ='Zip', how ='left')\
    .merge(maryland.zip_population.assign(Zip = lambda d: d.Zip.astype(int)), on ='Zip')\
    .pipe(lambda d: d[d.Date==d.Date.max()])  \
    .filter(['Zip','City','State','Cases','Date','geometry', 'Population']) \
    .assign(Cases = lambda d: d.Cases.fillna(0)) \
    .pipe(lambda d: d[~pd.isnull(d.geometry)]) \
    .assign(per_population = lambda d: d.Cases / d.Population.astype(int) * 1000)
data.head()

,Zip,City,State,Cases,Date,geometry,Population,per_population
2,20910,Silver Spring,MD,97.0,2020-04-14,"POLYGON ((-77.06795 39.02395, -77.06792 39.023...",42868,2.262760
4,20912,Takoma Park,MD,46.0,2020-04-14,"POLYGON ((-77.02669 38.98656, -77.02644 38.986...",26239,1.753116
8,21009,Abingdon,MD,14.0,2020-04-14,"POLYGON ((-76.34463 39.48586, -76.34453 39.485...",29997,0.466713
11,21012,Arnold,MD,52.0,2020-04-14,"POLYGON ((-76.54527 39.04052, -76.54289 39.042...",21772,2.388389
14,21014,Bel Air,MD,17.0,2020-04-14,"POLYGON ((-76.39499 39.53694, -76.39495 39.537...",35681,0.476444


In [4]:
geosource = GeoJSONDataSource(geojson = data.drop('Date', axis=1).to_json())
color_mapper = LinearColorMapper(palette=Inferno256, 
                           low=data.per_population.min(), 
                           high=data.per_population.max())

p = figure(title = 'COVID19 in Maryland (Cases per 1,000 people)', 
           plot_height = 600,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
p.title.text_font_size = '15pt'

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field':'per_population',
                                'transform':color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p.add_tools(HoverTool(tooltips = [('City','@City'),
                                  ('Zip code','@Zip'),
                                  ('Population','@Population'),
                                  ('Cases', '@Cases'),
                                  ('Cases/1k population', '@per_population')]))

#color bar
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                    orientation='horizontal')
p.add_layout(color_bar, 'below')
output_file('output.html')
save(p)


INFO:bokeh.io.state:Session output file 'output.html' already exists, will be overwritten.


'/home/wckdouglas/codes/covid/output.html'

In [7]:
markdown_html('output.html','COVID.html')

INFO:Data collector:Written 84 lines from 85 lines to COVID.html
